In [1]:
#import dictionary
import json
import pandas as pd

# csv and json file path 
path_input='C:/Users/Andres Camilo Guzman/Documents/Trebol/MP/first50.json'
path_output_json='C:/Users/Andres Camilo Guzman/Documents/Trebol/MP/result-first50.json'
path_output_csv='C:/Users/Andres Camilo Guzman/Documents/Trebol/MP/result-first50.csv'

#import json input as list
with open(path_input, encoding="utf8") as json_file:
    all_results = json.load(json_file).get('results')

#standardization dictionaries
currency_dict = {'ARS':'ARS', 'BRL':'BRL', 'CLP':'CLP', 'MXN':'MXN', 'COP':'COP', 'PEN':'PEN', 'UYU':'UYU'}
payment_status_dict={'pending':'pending', 'approved':'paid', 'authorized':'authorized', 'in_process':'unknown',
                     'in_mediation':'unknown', 'rejected':'failed', 'cancelled':'canceled', 'refunded':'refunded',
                     'charged_back':'charged_back'}
payment_method_dict={'credit_card':'credit_card', 'account_money':'store_credit', 'atm':'cash', 
                     'prepaid_card':'prepaid_card','debit_card':'debit_card', 'ticket':'cash', 
                     'bank_transfer':'bank_transfer'}

#standardization function
def converter (results,dict_output):
   
    #is not an error when payer element exists in first level of trx dictionary
    try:
        # extract data from input dict and standardize 
        status=payment_status_dict.get(results.get('status'))
        customerName=results.get('payer').get('email')
        amount=results.get('transaction_amount')
        currency=currency_dict.get(results.get('currency_id'))
        dueDate=results.get('money_release_date')
        createdDate=results.get('date_created')
        source_modified_date=results.get('date_last_updated')
        customerID=results.get('payer').get('id')
        transactionId=results.get('id')
        paymentMethod=payment_method_dict.get(results.get('payment_type_id'))
        operatyonType=results.get('operation_type')
        net_amount=results.get('transaction_details').get('net_received_amount')
        if (status=='refunded'):
            #create cashin trx
            dict_name={'customerName':customerName, 'customerID':customerID}
            dict_trx={'amount':amount, 'currency':currency, 'status':status, 'dueDate':dueDate, 'createdDate':createdDate,
                    'source_modified_date':source_modified_date, 'customer':dict_name, 'paymentMethod':paymentMethod, 'operationType':operatyonType,'transactionID':transactionId,'net_amount':net_amount}
            dict_output.append(dict_trx)
            #get refund info
            refunds_dict=results.get('refunds')[0]
            amount_refund=refunds_dict.get('amount')
            createdDate_refund=refunds_dict.get('date_created')
            transactionID_refund=refunds_dict.get('id')
            status_refund=payment_status_dict.get(refunds_dict.get('status'))
            #create refund trx
            dict_trx={'amount':-amount_refund, 'currency':currency, 'status':status_refund, 'dueDate':'NaN', 'createdDate':createdDate_refund,
                    'source_modified_date':'NaN', 'customer':dict_name, 'paymentMethod':paymentMethod, 'operationType':operatyonType,'transactionID':transactionID_refund,'net_amount':'NaN'}
            dict_output.append(dict_trx)
        else:    
            # create output dict
            dict_name={'customerName':customerName, 'customerID':customerID}
            dict_trx={'amount':amount, 'currency':currency, 'status':status, 'dueDate':dueDate, 'createdDate':createdDate,
                    'source_modified_date':source_modified_date, 'customer':dict_name, 'paymentMethod':paymentMethod, 'operationType':operatyonType,'transactionID':transactionId,'net_amount':net_amount}
            dict_output.append(dict_trx)
        return dict_output
    #if not found catch error and return output_dict unchanged
    except:
        return dict_output

# declare output list of dicts
dict_output=[]

#iterate over trx list to standardize 
for i in range(0,len(all_results)-1):
    dict_output=converter(all_results[i], dict_output)

# dump output dict into json file
with open(path_output_json, 'w') as fp:
    json.dump(dict_output, fp)

In [2]:
df_output=pd.read_json(path_output_json)

In [ ]:
df_output.to_csv(path_output_csv)

In [6]:
df_output.columns

Index(['amount', 'currency', 'status', 'dueDate', 'createdDate',
       'source_modified_date', 'customer', 'paymentMethod', 'operationType',
       'transactionID', 'net_amount'],
      dtype='object')

In [12]:

type(df_output['createdDate'][0])

str